In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langgraph.graph import StateGraph, END
# 状態管理用のデータ構造
from pydantic import BaseModel,Field
from typing import Any


/Users/sumiya/git/langgraph-agent/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
class AgentState(BaseModel):
    user_input: str = Field(...,description="user_input")
    response: str = Field(...,description="ai response")

In [3]:
# LLM のセットアップ
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
def call_llm(state: AgentState) -> dict[str,Any]:
    """LLM にユーザー入力を渡し、応答を取得"""
    prompt = ChatPromptTemplate.from_messages([
        ("human","{user_input}")
    ])
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({"user_input": state.user_input})
    return {"response": response}

In [8]:
# グラフの構築
workflow = StateGraph(AgentState)

# ノードの追加
workflow.add_node("llm", call_llm)

# フローの設定
workflow.set_entry_point("llm")
workflow.add_edge("llm", END)

# ワークフローの作成
app = workflow.compile()

In [9]:
state = AgentState(user_input="あなたは優秀なエージェントです。カレーのレシピを考えて下さい。",response="")
result = app.invoke(state)

In [10]:
print("エージェントの応答:", result["response"])

エージェントの応答: ありがとうございます！ここにシンプルでおいしいカレーのレシピをご紹介します。日本風のカレーを作るための基本的なレシピです。

### 材料（4人分）
- 鶏もも肉または牛肉：300g
- 玉ねぎ：2個
- にんじん：1本
- じゃがいも：2個
- にんにく：2片
- ショウガ：1片
- カレールー：1箱（約200g）
- サラダ油：大さじ2
- 水：800ml
- 塩、コショウ：少々
- ご飯：適量

### 作り方
1. **材料の準備**  
   - 鶏もも肉または牛肉を一口大に切り、塩コショウを振ります。
   - 玉ねぎは薄切り、にんじんは半月切り、じゃがいもは一口大に切ります。
   - にんにくとショウガはみじん切りにします。

2. **炒める**  
   - 大きな鍋にサラダ油を熱し、にんにくとショウガを香りが立つまで炒めます。
   - 肉を加え、表面が白くなるまで炒めます。
   - 玉ねぎ、にんじん、じゃがいもを加え、玉ねぎが透明になるまで炒めます。

3. **煮込む**  
   - 水を加え、沸騰したらアクを取り除きます。
   - 弱火にし、蓋をして20分ほど煮込みます。

4. **カレールーを加える**  
   - 火を止めて、カレールーを少しずつ加えながら溶かします。
   - 再び弱火で煮込み、15分ほどとろみがつくまで煮ます。

5. **仕上げ**  
   - 味を見て、必要なら塩コショウで調整します。
   - ご飯を盛った皿にカレーをかけて、出来上がりです。

このレシピは基本ですが、お好みでリンゴやはちみつを入れると甘みが増し、スパイスを足すと風味が豊かになります。お楽しみください！
